In [9]:
'''from google.colab import files
uploaded = files.upload()'''

'from google.colab import files\nuploaded = files.upload()'

In [10]:
!nvidia-smi

Wed Jun 24 00:08:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
PATH = '/content/drive/My Drive/NLP/'

In [12]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
!pip install transformers
from transformers import *
import tokenizers
print('TF version',tf.__version__)
import re
import math
import keras
import json
!pip install emoji
import emoji
import string
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences

TF version 2.2.0


In [13]:
with open(PATH+'categories.json') as f:
  categories = json.load(f)

tweets = []
for line in open(PATH+'train_gold.json', 'r'):
    tweets.append(json.loads(line))

test = []
for line in open(PATH+'test_unlabeled.json', 'r'):
    test.append(json.loads(line))
    
dev = []
for line in open(PATH+'dev_unlabeled.json','r'):
  dev.append(json.loads(line))

In [14]:
f = open(PATH+"weird.txt")
weird = f.read().splitlines()
#print(weird)
f.close()

In [15]:
"""input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(input_ids, labels=labels)
loss, logits = outputs[:2]

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(input_ids, labels=labels)
loss, logits = outputs[:2]"""

'input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1\nlabels = torch.tensor([1]).unsqueeze(0)  # Batch size 1\noutputs = model(input_ids, labels=labels)\nloss, logits = outputs[:2]\n\ntokenizer = RobertaTokenizer.from_pretrained(\'roberta-base\')\nmodel = RobertaForSequenceClassification.from_pretrained(\'roberta-base\')\ninput_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1\nlabels = torch.tensor([1]).unsqueeze(0)  # Batch size 1\noutputs = model(input_ids, labels=labels)\nloss, logits = outputs[:2]'

In [16]:
def rule(token):
    '''
    Remove tokens with
    x. stop words  Ex. the, we (Update: only rmv '️')
    2. punctuation Ex. !!
    3. numbers     Ex. 1kg
    4. hashtags    Ex. #NLP
    5. tags        Ex. @hannnnk
    6. hyper link  Ex. https://...
    '''
    return True if token not in stopword and \
                   token not in punctuation and \
                   "\n" not in token and \
                   not re.search('\d+', token) and \
                   not token.startswith('#') and \
                   not token.startswith('@') and \
                   not token.startswith('https:') \
                   else False

In [17]:
def clean(tweet):
    tweet = re.sub(r"what's", "what is", tweet)
    tweet = re.sub(r"we'll", "we will", tweet)
    tweet = re.sub(r"who's", "who is", tweet)
    tweet = re.sub(r"here's", "here is", tweet)
    tweet = re.sub(r"today's", "today is", tweet)
    tweet = re.sub(r"ain't", "are not", tweet)
    tweet = re.sub(r"we've", "we have", tweet)
    tweet = re.sub(r"how's", "how is", tweet)
    tweet = re.sub(r"thet've", "they have", tweet)
    tweet = re.sub(r"it'll", "it will", tweet)
    tweet = re.sub(r"ya'll", "you all", tweet)
    tweet = re.sub(r"i'm", "i am", tweet)
    tweet = re.sub(r"don't", "do not", tweet)
    tweet = re.sub(r"it's", "it is", tweet)
    tweet = re.sub(r"you're", "you are", tweet)
    tweet = re.sub(r"can't", "cannot", tweet)
    tweet = re.sub(r"i've", "i have", tweet)
    tweet = re.sub(r"that's", "that is", tweet)
    tweet = re.sub(r"we're", "we are", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"he's", "he is", tweet)
    tweet = re.sub(r"let's", "let us", tweet)
    tweet = re.sub(r"didn't", "did not", tweet)
    tweet = re.sub(r"y'all", "you all", tweet)
    tweet = re.sub(r"doesn't", "does not", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"who-the-hell-even-knows-anymore", "who the hell even knows anymore", tweet)
    tweet = re.sub(r"there's", "there is", tweet)
    tweet = re.sub(r"isn't", "is not", tweet)
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"they're", "they are", tweet)
    tweet = re.sub(r"haven't", "have not", tweet)
    tweet = re.sub(r"couldn't", "could not", tweet)
    tweet = re.sub(r"she's", "she is", tweet)
    tweet = re.sub(r"aren't", "are not", tweet)
    tweet = re.sub(r"you've", "you have", tweet)
    tweet = re.sub(r"you'll", "you will", tweet)
    tweet = re.sub(r"wouldn't", "would not", tweet)
    tweet = re.sub(r"wasn't", "was not", tweet)
    tweet = re.sub(r"shouldn't", "shoud not", tweet)  
    tweet = re.sub(r"they'll", "they will", tweet)
    tweet = re.sub(r"weren't", "were not", tweet)
    tweet = re.sub(r"you'd", "you had", tweet)
    tweet = re.sub(r"they've", "they have", tweet)
    tweet = re.sub(r"no-excuse", "no excuse", tweet)
    tweet = re.sub(r"vote-by-mail", "vote by mail", tweet)
    tweet = re.sub(r"votesafe", "vote safe", tweet)
    tweet = re.sub(r"lmaoooo", "laugh my ass off", tweet)
    tweet = re.sub(r"we'd", "we had", tweet)
    tweet = re.sub(r"he'll", "he will", tweet)
    tweet = re.sub(r"he'd", "he had", tweet)
    tweet = re.sub(r"would've", "would have", tweet)
    tweet = re.sub(r"hadn't", "had not", tweet)
    tweet = re.sub(r"diefrom", "die from", tweet)
    tweet = re.sub(r"dababy", "rapper", tweet)
    tweet = re.sub(r"hasn't", "has not", tweet)
    tweet = re.sub(r"testing-based", "testing based", tweet)
    tweet = re.sub(r"that'll", "that will", tweet)
    tweet = re.sub(r"should've", "should have", tweet)
    tweet = re.sub(r"cashapp", "cash app", tweet)
    tweet = re.sub(r"brexit-induced", "brexit induced", tweet)
    tweet = re.sub(r"president-hating", "president hating", tweet)
    tweet = re.sub(r"picked-over", "picked over", tweet)
    
    tweet = re.sub(r"what’s", "what is", tweet)
    tweet = re.sub(r"we’ll", "we will", tweet)
    tweet = re.sub(r"who’s", "who is", tweet)
    tweet = re.sub(r"here’s", "here is", tweet)
    tweet = re.sub(r"today’s", "today is", tweet)
    tweet = re.sub(r"ain’t", "are not", tweet)
    tweet = re.sub(r"we’ve", "we have", tweet)
    tweet = re.sub(r"how’s", "how is", tweet)
    tweet = re.sub(r"thet’ve", "they have", tweet)
    tweet = re.sub(r"it’ll", "it will", tweet)
    tweet = re.sub(r"ya’ll", "you all", tweet)
    tweet = re.sub(r"i’m", "i am", tweet)
    tweet = re.sub(r"don’t", "do not", tweet)
    tweet = re.sub(r"it’s", "it is", tweet)
    tweet = re.sub(r"you’re", "you are", tweet)
    tweet = re.sub(r"can’t", "cannot", tweet)
    tweet = re.sub(r"i’ve", "i have", tweet)
    tweet = re.sub(r"that’s", "that is", tweet)
    tweet = re.sub(r"we’re", "we are", tweet)
    tweet = re.sub(r"i’ll", "I will", tweet)
    tweet = re.sub(r"he’s", "he is", tweet)
    tweet = re.sub(r"let’s", "let us", tweet)
    tweet = re.sub(r"didn’t", "did not", tweet)
    tweet = re.sub(r"y’all", "you all", tweet)
    tweet = re.sub(r"doesn’t", "does not", tweet)
    tweet = re.sub(r"i’d", "I would", tweet)
    tweet = re.sub(r"there’s", "there is", tweet)
    tweet = re.sub(r"isn’t", "is not", tweet)
    tweet = re.sub(r"won’t", "will not", tweet)
    tweet = re.sub(r"they’re", "they are", tweet)
    tweet = re.sub(r"haven’t", "have not", tweet)
    tweet = re.sub(r"couldn’t", "could not", tweet)
    tweet = re.sub(r"she’s", "she is", tweet)
    tweet = re.sub(r"aren’t", "are not", tweet)
    tweet = re.sub(r"you’ve", "you have", tweet)
    tweet = re.sub(r"you’ll", "you will", tweet)
    tweet = re.sub(r"wouldn’t", "would not", tweet)
    tweet = re.sub(r"wasn’t", "was not", tweet)
    tweet = re.sub(r"shouldn’t", "shoud not", tweet)
    tweet = re.sub(r"they’ll", "they will", tweet)
    tweet = re.sub(r"weren’t", "were not", tweet)
    tweet = re.sub(r"you’d", "you had", tweet)
    tweet = re.sub(r"they’ve", "they have", tweet)
    tweet = re.sub(r"we’d", "we had", tweet)
    tweet = re.sub(r"he’ll", "he will", tweet)
    tweet = re.sub(r"he’d", "he had", tweet)
    tweet = re.sub(r"would’ve", "would have", tweet)
    tweet = re.sub(r"hadn’t", "had not", tweet)
    tweet = re.sub(r"hasn’t", "has not", tweet)
    tweet = re.sub(r"that’ll", "that will", tweet)
    tweet = re.sub(r"should’ve", "should have", tweet)
    
    return tweet

In [18]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
tknzr = TweetTokenizer()

In [19]:
stopword = ['️']
punctuation = string.punctuation+'“’”...—…‼‘'

In [20]:
for t in tweets:
    t['text'] = clean(t['text'].lower())
    t['reply'] = clean(t['reply'].lower())
for t in dev:
    t['text'] = clean(t['text'].lower())
    t['reply'] = clean(t['reply'].lower())
for t in test:
    t['text'] = clean(t['text'].lower())
    t['reply'] = clean(t['reply'].lower())

In [21]:
for t in tweets:
    temp = [token.lower() for token in tknzr.tokenize(t['text']) if rule(token.lower())]
    temp2 = [token.lower() for token in tknzr.tokenize(t['reply']) if rule(token.lower())]

    t['text_clean'] = temp
    t['reply_clean'] = temp2

for t in dev:
    temp = [token.lower() for token in tknzr.tokenize(t['text']) if rule(token.lower())]
    temp2 = [token.lower() for token in tknzr.tokenize(t['reply']) if rule(token.lower())]

    t['text_clean'] = temp
    t['reply_clean'] = temp2
    
for t in test:
    temp = [token.lower() for token in tknzr.tokenize(t['text']) if rule(token.lower())]
    temp2 = [token.lower() for token in tknzr.tokenize(t['reply']) if rule(token.lower())]

    t['text_clean'] = temp
    t['reply_clean'] = temp2

In [22]:
'''
Deal with special words for embedding such as
covid  (806) → coronavirus
'''
special = {
    'covid':'coronavirus',
    "ventilators":"ventilator",
    "virologist":'virology',
    'fauci':'physician',
    "onlyfans":"app",
    "fortnite":"game",
    "valorant":"game",
    "jeudy":"athlete",
    "ruggs":"athlete",
    "tagovailoa":"athlete",
    "rashford":"athlete",
    "birx":"doctor",
    "blaqbonez":"rapper",
    'mcsally':"politician",
    "intubated":"intubation",
    "hydroxychloroquine":"medication",
    "today's":"today",
    "pubg":"game",
    "government's":'government',
    "man's":'man',
    "someone's":'someone',
    "country's":'country',
    "everyone's":'everyone',
    "people's":'people',
    "club's":'club',
    "china's":'china',
    "america's":'america',
    "world's":'world',
    "life's":'life',
    "earth's":'earth',
    "mom's":'mom',
    '<3':'❤',
    ':)':"🙂",
    ":(":"😞",
    "):":"😞",
    ":/":"😕",
    "pre-covid":'coronavirus',
    "strategise":"strategies",
    'stratagy':'strategy',
    "strategiser":"strategy",
    "stratagised":"strategy",
    'ohhhh':"ohhh",
    'yesssss':"yes",
    'youuu':'you',
    'sirrr':'sir',
    'ohhhhh':'ohhh',
    'youuuu':'you',
    'lmaooooo':'lamo',
    'guysss':'guys',
    'meeeee':'me',
    'yessss':'yes',
    'lieeeeeee':'lie',
    'youuuuu':'you',
    'yesssssss':'yes',
    'lollll':'lol',
    'plzzzz':'plz',
    'enddddddd':'end',
}

In [23]:
emoji_remain_text = ['😭','❤','👏','😊','😔','🙏','🙃','💜','👍','🙌','😁','🤗','💔','☹','😎','😢','💛','💖','💰','💯','💗','💪','😞','😣',"🙂","😞","😕"]
emoji_remain_reply = ['❤','🙏','😭','👏','💜','👍','🤗','💕','💖','😘','💙','♥','💞','🙌','💗','😉','💛','🤦','💓','🖤','🙄','💯','☺','💃','🎉','😔','💚','💔','✨','😢','🌹',"🙂","😞","😕"]

In [24]:
def rule_emoji_text(token, emoji_remain):
    if len(token)==1:
        if ord(token)>122:
            if token not in emoji_remain:
              return False
    return True

In [25]:
tokens = Counter()
tokens2 = Counter()
for t in tweets:
    temp = [token for token in t['text_clean'] if rule_emoji_text(token,emoji_remain_text)]
    t['text_clean'] = temp
    tokens.update(temp)
    temp2 = [token for token in t['reply_clean'] if rule_emoji_text(token,emoji_remain_reply)]
    for idx,token in enumerate(temp2):
        if token == '🥺':
            temp2[idx] = '❤'
    t['reply_clean'] = temp2
    tokens2.update(temp2)
    
temp = pd.DataFrame(tokens.most_common(30))
temp.columns = ['tokens_main','count']
display(temp.style.background_gradient(cmap='Blues'))
temp2 = pd.DataFrame(tokens2.most_common(30))
temp2.columns = ['tokens_reply','count']
display(temp2.style.background_gradient(cmap='Reds'))

for t in dev:
    temp = [token for token in t['text_clean'] if rule_emoji_text(token,emoji_remain_text)]
    t['text_clean'] = temp
    temp2 = [token for token in t['reply_clean'] if rule_emoji_text(token,emoji_remain_reply)]
    for idx,token in enumerate(temp2):
        if token == '🥺':
            temp2[idx] = '❤'
    t['reply_clean'] = temp2
    
for t in test:
    temp = [token for token in t['text_clean'] if rule_emoji_text(token,emoji_remain_text)]
    t['text_clean'] = temp
    temp2 = [token for token in t['reply_clean'] if rule_emoji_text(token,emoji_remain_reply)]
    for idx,token in enumerate(temp2):
        if token == '🥺':
            temp2[idx] = '❤'
    t['reply_clean'] = temp2

,tokens_main,count
0,i,22433
1,the,18746
2,to,17175
3,a,14900
4,and,12369
5,is,12307
6,you,11669
7,of,9100
8,my,7713
9,in,7540


,tokens_reply,count
0,i,3620
1,you,3411
2,the,2588
3,to,2197
4,is,2025
5,a,1952
6,and,1819
7,it,1562
8,not,1340
9,that,1256


In [26]:
for t in tweets:
    for idx,token in enumerate(t['text_clean']):
        if token in special:
            t['text_clean'][idx] = special[token]
    for idx,token in enumerate(t['reply_clean']):
        if token in special:
            t['reply_clean'][idx] = special[token]
    t['text_clean'] = [c for c in t['text_clean'] if c not in weird]
    t['reply_clean'] = [c for c in t['reply_clean'] if c not in weird]
for t in dev:
    for idx,token in enumerate(t['text_clean']):
        if token in special:
            t['text_clean'][idx] = special[token]
    for idx,token in enumerate(t['reply_clean']):
        if token in special:
            t['reply_clean'][idx] = special[token]
    t['text_clean'] = [c for c in t['text_clean'] if c not in weird]
    t['reply_clean'] = [c for c in t['reply_clean'] if c not in weird]
for t in test:
    for idx,token in enumerate(t['text_clean']):
        if token in special:
            t['text_clean'][idx] = special[token]
    for idx,token in enumerate(t['reply_clean']):
        if token in special:
            t['reply_clean'][idx] = special[token]
    t['text_clean'] = [c for c in t['text_clean'] if c not in weird]
    t['reply_clean'] = [c for c in t['reply_clean'] if c not in weird]

Deomojize Mode

In [27]:
'''for t in tweets:
    for idx,token in enumerate(t['text_clean']):
        if token in special:
            t['text_clean'][idx] = special[token]
        if token in emoji_remain_text:
            t['text_clean'][idx] = re.sub("_", " ",re.sub(':','',emoji.demojize(token)))
    for idx,token in enumerate(t['reply_clean']):
        if token in special:
            t['reply_clean'][idx] = special[token]
        if token in emoji_remain_reply:
            t['reply_clean'][idx] = re.sub("_", " ",re.sub(':','',emoji.demojize(token)))
    #t['text_clean'] = [c for c in t['text_clean'] if c not in weird]
    #t['reply_clean'] = [c for c in t['reply_clean'] if c not in weird]
for t in dev:
    for idx,token in enumerate(t['text_clean']):
        if token in special:
            t['text_clean'][idx] = special[token]
        if token in emoji_remain_text:
            t['text_clean'][idx] = re.sub("_", " ",re.sub(':','',emoji.demojize(token)))
    for idx,token in enumerate(t['reply_clean']):
        if token in special:
            t['reply_clean'][idx] = special[token]
        if token in emoji_remain_reply:
            t['reply_clean'][idx] = re.sub("_", " ",re.sub(':','',emoji.demojize(token)))
    #t['text_clean'] = [c for c in t['text_clean'] if c not in weird]
    #t['reply_clean'] = [c for c in t['reply_clean'] if c not in weird]
for t in test:
    for idx,token in enumerate(t['text_clean']):
        if token in special:
            t['text_clean'][idx] = special[token]
        if token in emoji_remain_text:
            t['text_clean'][idx] = re.sub("_", " ",re.sub(':','',emoji.demojize(token)))
    for idx,token in enumerate(t['reply_clean']):
        if token in special:
            t['reply_clean'][idx] = special[token]
        if token in emoji_remain_reply:
            t['reply_clean'][idx] = re.sub("_", " ",re.sub(':','',emoji.demojize(token)))
    #t['text_clean'] = [c for c in t['text_clean'] if c not in weird]
    #t['reply_clean'] = [c for c in t['reply_clean'] if c not in weird]'''

'for t in tweets:\n    for idx,token in enumerate(t[\'text_clean\']):\n        if token in special:\n            t[\'text_clean\'][idx] = special[token]\n        if token in emoji_remain_text:\n            t[\'text_clean\'][idx] = re.sub("_", " ",re.sub(\':\',\'\',emoji.demojize(token)))\n    for idx,token in enumerate(t[\'reply_clean\']):\n        if token in special:\n            t[\'reply_clean\'][idx] = special[token]\n        if token in emoji_remain_reply:\n            t[\'reply_clean\'][idx] = re.sub("_", " ",re.sub(\':\',\'\',emoji.demojize(token)))\n    #t[\'text_clean\'] = [c for c in t[\'text_clean\'] if c not in weird]\n    #t[\'reply_clean\'] = [c for c in t[\'reply_clean\'] if c not in weird]\nfor t in dev:\n    for idx,token in enumerate(t[\'text_clean\']):\n        if token in special:\n            t[\'text_clean\'][idx] = special[token]\n        if token in emoji_remain_text:\n            t[\'text_clean\'][idx] = re.sub("_", " ",re.sub(\':\',\'\',emoji.demojize(token))

In [28]:
cat_index = {}
for idx,c in enumerate(categories):
    cat_index[c]=idx
    
def cat2vec(cat):
    res = []
    for tweet in cat:
        temp = np.zeros(43)
        for c in tweet:
            temp[cat_index[c]] = 1
        res.append(temp)
    return np.array(res)

In [29]:
tweets[4579]['text_clean'] = tweets[4579]['text_clean'][:20]+['what','is','the','most','shocking','book','you','were','forced','to','read','in','high','school']
tweets[10415]['reply_clean'] = tweets[10415]['reply_clean'][37:]
tweets[25505]['text_clean'] = tweets[25505]['text_clean'][:14]
tweets[28767]['reply_clean'] = tweets[28767]['reply_clean'][:63]
train_text = [" ".join(t['text_clean']) for t in tweets]
train_reply = [" ".join(t['reply_clean']) for t in tweets]
train_y = [t['categories'] for t in tweets]

dev_text = [" ".join(t['text_clean']) for t in dev]
dev_reply = [" ".join(t['reply_clean']) for t in dev]

test[1487]['text_clean'] = test[1487]['text_clean'][5:]
test[2949]['reply_clean'] = test[2949]['reply_clean'][:11]
test_text = [" ".join(t['text_clean']) for t in test]
test_reply = [" ".join(t['reply_clean']) for t in test]

In [30]:
train_y = cat2vec(train_y)
train_y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [31]:
train_dict = {}
for i in range(32000):
  temp = train_text[i]+train_reply[i]
  if temp in train_dict:
    train_dict[temp].append((train_y[i],i))
  else:
    train_dict[temp] = [(train_y[i],i)]

duplicated = []
for key in train_dict:
  if len(train_dict[key])>1:
    for id in train_dict[key][1:]:
      duplicated.append(id[1])

new = []
for key in train_dict:
  if len(train_dict[key])>1:
    first_idx = train_dict[key][0][1]
    temp = np.zeros(43)
    for id in train_dict[key]:
      temp = np.logical_or(temp, id[0])
    temp = temp.astype(int)
    new.append((first_idx,temp))

In [32]:
for idx, cat in new:
  train_y[idx] = cat

In [33]:
print('before: ',len(train_text), len(train_reply), len(train_y))
_tweets = np.delete(tweets, duplicated)
train_text = np.delete(train_text, duplicated)
train_reply = np.delete(train_reply, duplicated)
train_y = np.delete(train_y, duplicated, axis=0)
print('after: ',len(train_text), len(train_reply), len(train_y))

before:  32000 32000 32000
after:  26784 26784 26784


In [34]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
#tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')
#tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

In [35]:
train_x = []
for i in range(len(train_text)):
  if train_reply[i] == '':
    if train_text[i] == '':
      train_x.append(tokenizer.encode(' ',' '))
    else:
      train_x.append(tokenizer.encode(train_text[i],' ',skip_special_tokens=True))
  else:
    if train_text[i] == '':
      train_x.append(tokenizer.encode(' ', train_reply[i],skip_special_tokens=True))
    else:
      train_x.append(tokenizer.encode(train_text[i], train_reply[i],skip_special_tokens=True))

dev_x = []
for i in range(len(dev_text)):
  if dev_reply[i] == '':
    dev_x.append(tokenizer.encode(dev_text[i],skip_special_tokens=True))
  else:
    dev_x.append(tokenizer.encode(dev_text[i], dev_reply[i],skip_special_tokens=True))

test_x = []
for i in range(len(test_text)):
  if test_reply[i] == '':
    if test_text[i] == '':
      test_x.append(tokenizer.encode(' ',' '))
    else:
      test_x.append(tokenizer.encode(test_text[i],' ',skip_special_tokens=True))
  else:
    if test_text[i] == '':
      test_x.append(tokenizer.encode(' ', test_reply[i],skip_special_tokens=True))
    else:
      test_x.append(tokenizer.encode(test_text[i], test_reply[i],skip_special_tokens=True))

In [36]:
print(len(max(train_x,key=len)))
print(len(max(dev_x,key=len)))
print(len(max(test_x,key=len)))

207
157
156


In [37]:
gg = []
for idx,t in enumerate(test_x):
  gg.append((len(t),idx))
sorted(gg,key=lambda x:x[0],reverse=True)[:5]

[(156, 1487), (141, 1987), (131, 645), (116, 3490), (113, 2504)]

In [38]:
MAX_LEN = 227
#MAX_LEN = 133

In [39]:
tokenizer.decode(train_x[0])

'<s> we can all agree that any song by niall horan</s></s> oui oui</s>'

In [40]:
attention_mask = []
for t in train_x:
  N = len(t)
  attention_mask.append(list(np.ones(N, dtype=int))+list(np.zeros(MAX_LEN-N, dtype=int)))

attention_mask_d = []
for t in dev_x:
  N = len(t)
  attention_mask_d.append(list(np.ones(N, dtype=int))+list(np.zeros(MAX_LEN-N, dtype=int)))

attention_mask_t = []
for t in test_x:
  N = len(t)
  attention_mask_t.append(list(np.ones(N, dtype=int))+list(np.zeros(MAX_LEN-N, dtype=int)))

In [41]:
attention_mask = np.mat(attention_mask)
attention_mask_d = np.mat(attention_mask_d)
attention_mask_t = np.mat(attention_mask_t)

In [42]:
train_x = pad_sequences(train_x,  maxlen=MAX_LEN, dtype='int', padding='post', value = 1)
dev_x = pad_sequences(dev_x,  maxlen=MAX_LEN, dtype='int', padding='post', value = 1)
test_x = pad_sequences(test_x,  maxlen=MAX_LEN, dtype='int', padding='post', value = 1)

In [43]:
token_type_ids = np.zeros((train_x.shape[0],MAX_LEN))
token_type_ids_d = np.zeros((dev_x.shape[0],MAX_LEN))
token_type_ids_t = np.zeros((test_x.shape[0],MAX_LEN))

## Build roBERTa Model

In [44]:
def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    config = RobertaConfig.from_pretrained('roberta-base', num_labels=43)
    bert_model = TFRobertaForSequenceClassification.from_pretrained('roberta-base',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x1 = tf.keras.layers.Dense(43)(x1)
    x1 = tf.keras.layers.Activation('sigmoid')(x1)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=x1)
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)

    return model

In [45]:
def build_model_large():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    config = RobertaConfig.from_pretrained('roberta-large', num_labels=43)
    bert_model = TFRobertaForSequenceClassification.from_pretrained('roberta-large',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x1 = tf.keras.layers.Dense(43)(x1)
    x1 = tf.keras.layers.Activation('sigmoid')(x1)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=x1)
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)

    return model

In [46]:
def build_model_distil():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    config = RobertaConfig.from_pretrained('distilroberta-base', num_labels=43)
    bert_model = TFRobertaForSequenceClassification.from_pretrained('distilroberta-base',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x1 = tf.keras.layers.Dense(43)(x1)
    x1 = tf.keras.layers.Activation('sigmoid')(x1)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=x1)
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)

    return model

### Roberta(Base)

In [ ]:
model = build_model()

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

model.fit([train_x, attention_mask, token_type_ids], train_y, epochs=5, batch_size=32, verbose=1)

### Boberta (Large)
> failed even remove emoji, maxlen=133, the preds are all same



In [ ]:
model_large = build_model_large()

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

model_large.fit([train_x, attention_mask, token_type_ids], train_y, epochs=4, batch_size=16, verbose=1)

### Pred and Save

In [ ]:
pred = model.predict([dev_x, attention_mask_d, token_type_ids_d])
pred_t = model.predict([test_x, attention_mask_t, token_type_ids_t])

In [ ]:
with open(PATH+'pred_roberta_duplicate2_d.npy', 'wb') as f:
    np.save(f, pred)

with open(PATH+'pred_roberta_duplicate2_t.npy', 'wb') as f:
    np.save(f, pred_t)

## Ensemble Roberta(Base)
* 0-16: batch size 32
* 16-21: batch size 16
* 22-27: batch size 16, epoch 7
* 28-31: demoji, batch size 16

In [ ]:
start = 22
for i in range(start,start+10):
  config = tf.compat.v1.ConfigProto()
  config.gpu_options.allow_growth = True
  model = build_model()
  model.fit([train_x, attention_mask, token_type_ids], train_y, epochs=7, batch_size=16, verbose=1)
  pred = model.predict([dev_x, attention_mask_d, token_type_ids_d])
  pred_t = model.predict([test_x, attention_mask_t, token_type_ids_t])
  with open(PATH+'pred_roberta_duplicate{}_d.npy'.format(i), 'wb') as f:
    np.save(f, pred)

  with open(PATH+'pred_roberta_duplicate{}_t.npy'.format(i), 'wb') as f:
    np.save(f, pred_t)
  del model

In [75]:
pred = []
pred_t = []
start = 0
end = 31
for i in range(start, end+1):
  with open(PATH+'pred_roberta_duplicate{}_d.npy'.format(i), 'rb') as f:
    a = np.load(f)
    pred.append(a)
  with open(PATH+'pred_roberta_duplicate{}_t.npy'.format(i), 'rb') as f:
    a = np.load(f)
    pred_t.append(a)
pred = np.mean(pred,axis=0)
pred_t = np.mean(pred_t,axis=0)

with open(PATH+'pred_roberta_ensemble{}-{}_d.npy'.format(start,end), 'wb') as f:
    np.save(f, pred)

with open(PATH+'pred_roberta_ensemble{}-{}_t.npy'.format(start,end), 'wb') as f:
    np.save(f, pred_t)

## Ensemble Distilroberta

In [ ]:
start = 6
for i in range(start,start+10):
  config = tf.compat.v1.ConfigProto()
  config.gpu_options.allow_growth = True
  model = build_model_distil()
  model.fit([train_x, attention_mask, token_type_ids], train_y, epochs=5, batch_size=32, verbose=1)
  pred = model.predict([dev_x, attention_mask_d, token_type_ids_d])
  pred_t = model.predict([test_x, attention_mask_t, token_type_ids_t])
  with open(PATH+'pred_distilroberta{}_d.npy'.format(i), 'wb') as f:
    np.save(f, pred)

  with open(PATH+'pred_distilroberta{}_t.npy'.format(i), 'wb') as f:
    np.save(f, pred_t)
  del model


Epoch 1/5
843/843 [==============================] - 378s 449ms/step - loss: 0.2094
Epoch 2/5
843/843 [==============================] - 378s 448ms/step - loss: 0.1604
Epoch 3/5
843/843 [==============================] - 377s 448ms/step - loss: 0.1502
Epoch 4/5
843/843 [==============================] - 377s 447ms/step - loss: 0.1422
Epoch 5/5
843/843 [==============================] - 378s 448ms/step - loss: 0.1351
Epoch 1/5
843/843 [==============================] - 378s 449ms/step - loss: 0.2221
Epoch 2/5
843/843 [==============================] - 379s 449ms/step - loss: 0.1622
Epoch 3/5
843/843 [==============================] - 378s 449ms/step - loss: 0.1505
Epoch 4/5
843/843 [==============================] - 378s 449ms/step - loss: 0.1423
Epoch 5/5
843/843 [==============================] - 378s 448ms/step - loss: 0.1344
Epoch 1/5
843/843 [==============================] - 378s 449ms/step - loss: 0.2299
Epoch 2/5
843/843 [==============================] - 379s 449ms/step - loss

In [ ]:
pred = []
pred_t = []
start = 0
end = 15
for i in range(start, end+1):
  with open(PATH+'pred_distilroberta{}_d.npy'.format(i), 'rb') as f:
    a = np.load(f)
    pred.append(a)
  with open(PATH+'pred_distilroberta{}_t.npy'.format(i), 'rb') as f:
    a = np.load(f)
    pred_t.append(a)
pred = np.mean(pred,axis=0)
pred_t = np.mean(pred_t,axis=0)

with open(PATH+'pred_distilroberta_ensemble{}-{}_d.npy'.format(start,end), 'wb') as f:
    np.save(f, pred)

with open(PATH+'pred_distilroberta_ensemble{}-{}_t.npy'.format(start,end), 'wb') as f:
    np.save(f, pred_t)

## Get Result

In [76]:
def decode_cat(res):
    ans = []
    for r in res:
        temp = [categories[i] for i in r]
        ans.append(temp)
    return ans

In [77]:
results = [np.argsort(res)[::-1][:6] for res in pred]
results = decode_cat(results)

In [78]:
for idx,t in enumerate(dev):
    t['categories'] = results[idx]

In [79]:
dev2 = []
for line in open(PATH+'dev_unlabeled.json','r'):
    dev2.append(json.loads(line))
for idx,d in enumerate(dev):
    dev2[idx]['categories']=d['categories']

In [80]:
for i in dev2:
  with open('dev.json', 'a', encoding='utf-8') as f:
    json.dump(i, f, ensure_ascii=False)
    f.write('\n')